In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from sklearn.linear_model import LinearRegression

def get_tita0(data):
    try:
        return data["θr"].iloc[0]
    except KeyError:
        return data["θ"].iloc[0]

def get_amplitud(data):  # asumo amplitud como Asen(θ)
    try:
        return (max(data["θr"]) - min(data["θr"])) / 2
    except KeyError:
        return (max(data["θ"]) - min(data["θ"])) / 2

def get_periodo(data):
    try:
        # Localizo un máximo y un mínimo
        maximo = data["θ"].idxmax()
        minimo = data["θ"].idxmin()
        # Calculo el periodo
        return abs((data["t"].iloc[maximo] - data["t"].iloc[minimo])) * 2 # *2 porque es de pico a pico
    except KeyError:
        # Localizo un máximo y un mínimo
        maximo = data["θr"].idxmax()
        minimo = data["θr"].idxmin()
        # Calculo el periodo
        return abs((data["t"].iloc[maximo] - data["t"].iloc[minimo])) * 2 # *2 porque es de pico a pico

def get_frecuencia(data):
    return 1 / get_periodo(data) # NO está en rad/sec, está en Hz

def format_pd(data, feature):
    data = data.dropna()
    # Arranca en t = 0
    print(data["t"].iloc[0])
    data["t"] = data["t"] - data["t"].iloc[0]
    # Selecciona la columna de interés
    try:
        data = data[["t", feature]]
    except KeyError:
        data = data[["t", "θ"]]
    return data

def get_largo(data):
    data_r = data["r"]
    return data_r.mean()

In [5]:
import os
import pandas as pd

# Adjust the input directory path to go up one level from the current directory
input_dir = os.path.join('..', 'datasets')

datasets = []

for folder in os.listdir(input_dir):
    folder_path = os.path.join(input_dir, folder)
    if os.path.isdir(folder_path):
        for file in os.listdir(folder_path):
            if file.endswith('.csv'):
                # Add dataset to the list with subfolder and folder
                datasets.append(os.path.join(folder, file))

dataset_params = []

for i, dataset in enumerate(datasets):
    # Load a specific dataset into a DataFrame
    DATASET_CHOSEN = i
    data = pd.read_csv(os.path.join(input_dir, datasets[DATASET_CHOSEN]))
    L = get_largo(data)
    data = format_pd(data, "θr")
    tita0, amplitud, T, frecuencia = get_tita0(data), get_amplitud(data), get_periodo(data), get_frecuencia(data)
    dataset_params.append((L, tita0, amplitud, T, frecuencia, (23 if dataset.split()[0] == "plata" else 72)))

0.834
2.436
1.101
1.635
1.101
1.869
1.735
1.335
1.034
1.769
1.268
1.669
1.601667
2.069
0.768
1.669
0.934
0.534
1.335
2.203


C:\Users\valen\AppData\Local\Temp\ipykernel_11696\1511929606.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["t"] = data["t"] - data["t"].iloc[0]
C:\Users\valen\AppData\Local\Temp\ipykernel_11696\1511929606.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["t"] = data["t"] - data["t"].iloc[0]


In [7]:
def graph_params(dataset_params):
    # Creo un DataFrame con los datos
    df = pd.DataFrame(dataset_params, columns=["L", "tita0", "A", "T", "f", "m"])

    # order the data by L and cut the first row
    df = df.sort_values(by="L")
    print(df.iloc[0])
    df = df.iloc[1:] # LE ESTOY SACANDO UN OUTLIER MAL DETECTADO

    # Fit a linear regression model to estimate gravity
    # Convierte a un array de 2D: reshape(-1, 1)
    reg = LinearRegression().fit(df["L"].values.reshape(-1, 1), (df["T"]**2).values.reshape(-1, 1))

    # Plot T^2 vs L
    plt.figure()
    plt.scatter(df["L"], df["T"]**2, color='blue', label='Data')
    x = np.linspace(df["L"].min(), df["L"].max(), 100)
    plt.plot(x, reg.predict(x.reshape(-1, 1)), color='red', label='Linear Fit')
    plt.xlabel('Length (L)')
    plt.ylabel('Period Squared (T^2)')
    plt.title('T^2 vs L')
    plt.legend()
    plt.show()

    # Estimate gravity (g) from the slope (T^2 = (4 * pi^2 / g) * L)
    slope = reg.coef_[0][0]
    g_estimated = (4 * np.pi**2 / slope) / 100 # cm to m
    print(f"Estimated gravity (g): {g_estimated} m/s^2")

    # T vs m
    plt.scatter(df["m"], df["T"])
    plt.xlabel("Masa (m)")
    plt.ylabel("T (s)")
    plt.show()

    # T vs tita0
    plt.scatter(df["tita0"], df["T"])
    plt.xlabel("θ0 (rad)")
    plt.ylabel("T (s)")
    plt.show()

    graph_params = df[["L", "tita0", "A", "T", "f", "m"]]
